In [1]:
import pandas as pd

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_7196/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [20]:
instruments = pd.read_csv("data/upstox/NSE.csv")
instruments

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
...,...,...,...,...,...,...,...,...,...,...,...,...
46188,NCD_FO|15778,15778.0,USDINR2461484.375PE,NaN,NaN,2024-06-14,84.375,0.25,1.0,OPTCUR,PE,NCD_FO
46189,NCD_FO|15779,15779.0,USDINR2461489.5CE,NaN,NaN,2024-06-14,89.500,0.25,1.0,OPTCUR,CE,NCD_FO
46190,NCD_FO|15780,15780.0,USDINR2461489.5PE,NaN,NaN,2024-06-14,89.500,0.25,1.0,OPTCUR,PE,NCD_FO
46191,NCD_FO|15781,15781.0,USDINR2461489.75CE,NaN,NaN,2024-06-14,89.750,0.25,1.0,OPTCUR,CE,NCD_FO


In [21]:
instruments = instruments.query('instrument_type == "EQUITY"')
instruments

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
...,...,...,...,...,...,...,...,...,...,...,...,...
6635,NSE_EQ|INE358U01012,11394.0,ZOTA,ZOTA HEALTH CARE LIMITED,485.00,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6636,NSE_EQ|INE840M01016,29050.0,ZUARI,ZUARI AGRO CHEMICALS LTD,199.90,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6637,NSE_EQ|INE217A01012,3827.0,ZUARIIND,ZUARI INDUSTRIES LIMITED,351.75,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6638,NSE_EQ|INE010B01027,7929.0,ZYDUSLIFE,ZYDUS LIFESCIENCES LTD,947.40,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [22]:
test = instruments.query('exchange != "NSE" and exchange !="BSE"')

In [23]:
test

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
...,...,...,...,...,...,...,...,...,...,...,...,...
6635,NSE_EQ|INE358U01012,11394.0,ZOTA,ZOTA HEALTH CARE LIMITED,485.00,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6636,NSE_EQ|INE840M01016,29050.0,ZUARI,ZUARI AGRO CHEMICALS LTD,199.90,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6637,NSE_EQ|INE217A01012,3827.0,ZUARIIND,ZUARI INDUSTRIES LIMITED,351.75,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6638,NSE_EQ|INE010B01027,7929.0,ZYDUSLIFE,ZYDUS LIFESCIENCES LTD,947.40,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [18]:
instruments_bse = instruments.query('exchange == "BSE_EQ"')
instruments_bse

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange


In [25]:
instruments_nse = instruments.query('exchange == "NSE_EQ"')
instruments_nse

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
...,...,...,...,...,...,...,...,...,...,...,...,...
6635,NSE_EQ|INE358U01012,11394.0,ZOTA,ZOTA HEALTH CARE LIMITED,485.00,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6636,NSE_EQ|INE840M01016,29050.0,ZUARI,ZUARI AGRO CHEMICALS LTD,199.90,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6637,NSE_EQ|INE217A01012,3827.0,ZUARIIND,ZUARI INDUSTRIES LIMITED,351.75,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
6638,NSE_EQ|INE010B01027,7929.0,ZYDUSLIFE,ZYDUS LIFESCIENCES LTD,947.40,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [28]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|INE404A01024/day/2024-01-05/2024-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"status":"success","data":{"candles":[["2024-01-05T00:00:00+05:30",478.95,480,473.05,475.55,120179,0],["2024-01-04T00:00:00+05:30",471,476.75,468.15,471.85,104859,0],["2024-01-03T00:00:00+05:30",472.9,477.35,466.2,469.25,93426,0],["2024-01-02T00:00:00+05:30",474.45,475.5,466,469.2,71316,0],["2024-01-01T00:00:00+05:30",472,477,468.35,474.45,76563,0]]}}
